<a href="https://colab.research.google.com/github/adrianmoses/text-search-nlp/blob/main/SemanticSearchNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers faiss torch faiss-gpu

     |████████████████████████████████| 89.7 MB 15 kB/s 


In [4]:
from transformers import AutoModel, AutoTokenizer
import torch
import faiss

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import json
with open('drive/MyDrive/sentences.json') as f:
    data = json.load(f)

In [7]:
data

['A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.',
 'The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century.',
 'Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.',
 'As of 2018, approximately 37.9 million people are infected with HIV globally.',
 'Cholera is an infection of the small intestine by some strains of the bacterium Vibrio cholerae.',
 'Classic cholera symptom is large amounts of watery diarrhea that lasts a few days. Vomiting and muscle cramps may also occur. Diarrhea can be so severe that it leads within hours to severe dehydration and electrolyte imbalance.',
 'The COVID-19 pandemic, also known as the coronavirus pandemic, is an ongoing pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome 

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
def encode_doc(document):
    vec = model(**tokenizer(document, return_tensors='pt'))[0].detach().squeeze()
    return torch.mean(vec, dim=0)

In [19]:
vectors = [encode_doc(sentence) for sentence in data]

In [20]:
import numpy as np
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))

index.add_with_ids(
    np.array([v.numpy() for v in vectors]),
    np.array(range(0, len(data)))
)

In [23]:
def search(query: str, k=1):
    encoded_query = encode_doc(query).unsqueeze(dim=0).numpy()
    top_k = index.search(encoded_query, k)
    scores = top_k[0][0]
    results = [data[_id] for _id in top_k[1][0]]
    return list(zip(results, scores))

In [24]:
search("How many people have died during Black Death?", k=5)

[('As of 2018, approximately 37.9 million people are infected with HIV globally.',
  52.61344),
 ('A pandemic is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people.',
  51.774704),
 ('The death toll of Spanish Flu is estimated to have been somewhere between 17 million and 50 million, and possibly as high as 100 million, making it one of the deadliest pandemics in human history.',
  51.187943),
 ('The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century.',
  50.574482),
 ('The Spanish flu, also known as the 1918 flu pandemic, was an unusually deadly influenza pandemic caused by the H1N1 influenza A virus.',
  47.01427)]